In [1]:
import os
import pydicom
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from ctviewer import CTViewer

In [3]:
def transform_to_hu(image):
    """
    Convert the pixel intensities to Hounsfield Units (HU) using RescaleSlope and RescaleIntercept
    """
    converted_image = image.pixel_array
    converted_image[converted_image <= -1000] = 0
    intercept = image.RescaleIntercept
    converted_image = image.RescaleSlope * converted_image.astype(np.float64)
    converted_image += np.int16(intercept)
    return converted_image

In [4]:
DATA_PATH = "./sample_data/"
dicom_filenames = os.listdir(DATA_PATH)
dicom_filepaths = [os.path.join(DATA_PATH, dicom_filename) for dicom_filename in dicom_filenames]

In [5]:
volumetric_image = np.zeros((len(dicom_filepaths), 512, 512))
instance_numbers = np.array([0 for i in range(len(dicom_filepaths))])

for i, dicom_filepath in enumerate(dicom_filepaths):
    dicom_file = pydicom.read_file(dicom_filepath)
    instance_number = int(dicom_file.InstanceNumber)
    img = transform_to_hu(dicom_file)
    volumetric_image[i, ] = img
    instance_numbers[i] = instance_number
    
indices = instance_numbers.argsort()
volumetric_image = volumetric_image[indices]

In [6]:
%matplotlib inline
CTViewer(volumetric_image)
plt.show()

interactive(children=(RadioButtons(description='Slice plane selection:', index=2, options=('y-z', 'z-x', 'x-y'…